<a href="https://colab.research.google.com/github/jsstar522/hunkim_ML/blob/master/02_NeuralNet_Backpropagation/01_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backpropagation

## Neural Network에서 계산량 구하기

Neural Network는 수많은 Layer와 노드(perceptron)으로 구성됩니다. 784개의 데이터를 Input layer에 집어넣고 10개 노드를 가진 1st hidden layer, 10개의 노드를 가진 2nd hidden layer가 있을 때, 필요한 weight와 bias는 몇개이며 계산량은 얼만큼 될까요?

![이미지](1.jpg)
*[그림1]하나의 노드에는 Input data만큼의 w(Weight)와 하나의 bias가 필요하다*

**이렇게 Neural Network는 엄청나게 많은 양의 가중치(Weight)가 필요합니다.** 계산량도 어마어마해질 뿐만 아니라 단순한 Machine learning에서 도출된 Cost Function (Error)처럼 식을 구하는 것조차 힘듭니다. Hypothesis를 통해 나온 1차 측정값 (representation)을 다시 새로운 2차 Hypothesis에 넣어야 하므로 엄청나게 복잡한 식이 나오겠죠. 이러한 복잡한 식으로 Error를 구하기는 불가능하다는 것을 *Marvin Minsky* 박사가 1969년에 증명했습니다. 이런 복잡한 식으로 미분을 해서 최소값을 찾아가는 과정...!@$%# 말이 안되죠. 이렇게 인공지능의 침체기가 시작됐지만 Backpropagation이 등장하면서 다시 불이 붙기 시작하죠.


## Backpropagation

**Backpropagation은 단순히 예측결과와 실제값의 차이만으로 오차를 줄여갑니다. 중간에 계산과정이 어떻게 됐던지 간에 오차가 줄어드는 방향을 근거 있게 설명합니다.**
바로 오류에 영향을 가장 크게 미치는 Weight값을 찾고 수정하는 것입니다. 

수많은 데이터가 수많은 다음 layer의 노드로 이동하는 과정 중에 하나만 확대해서 그려보겠습니다.

![이미지](2.jpg)
*[그림2]하나의 input data가 하나의 노드로 이동하는 과정*

이렇게 예측값이 나왔을 때 w(weight)가 예측값(f)에 미치는 영향이 얼마나 되는지 계산해보죠. **얼마나 영향을 미치는지 알아보려면 w값이 변할 때, f값은 얼마나 변하는지 구하면 됩니다. 즉, f를 w에 대해 편미분하면 되죠.**

$$\frac{⍺f}{⍺w}$$

f = g + b이므로,

$$\frac{⍺f}{⍺g}\frac{⍺g}{⍺w}$$

로 표현되어야 합니다. (이걸 미적분학에서 **Chain Rule**이라고 합니다.)

위 그림에서,
$$\frac{⍺f}{⍺g} = 1,\frac{⍺g}{⍺w} = x_{(x=5)}$$

이므로

$$\frac{⍺f}{⍺w} = 5$$

가 됩니다.

이 의미는 w가 1증가 할 때, f가 5배만큼 증가한다는 뜻이죠. w가 큰 영향을 미치는군요.

**같은 방식으로 f를 Error로 바꿔서 풀면 Error에 큰 영향을 미치는 w(weight)를 찾을 수 있게 됩니다. 그렇게 w를 "Optimizing" 해나가는 것입니다.**

![이미지][3.jpg]
*[그림3] 거꾸로 편미분해 나아가면 최초 w(weight) 값이 최종 f값에 얼마나 영향을 미치는지 알 수 있다.*

[그림3]을 보면 왜 Backpropagation이라는 이름이 붙었는지 느낌이 올겁니다. 

**이렇게 수많은 Layer를 거친 Hypothesis를 구해 복잡하게 미분을 하지 않고, 각 Layer에서의 편미분 값을 이용해 가중치(weight)의 영향력을 알 수 있습니다. 이 방법으로 Error에 미치는 영향력도 알 수 있고 Optimizing 하는 데 더 수월해졌습니다.**